In [1]:
import h5py
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.stats as ss

from uoineuro.utils import calculate_selection_ratio

%matplotlib inline

In [2]:
ret1_path = os.path.join(os.environ['HOME'],
                         'fits/uoineuro/tuning/ret1/')
ecog_path = os.path.join(os.environ['HOME'],
                         'fits/uoineuro/tuning/ecog/ecog_tuning.h5')

In [3]:
retinas = {
    '20080516_R1': ['cell0_recording0',
                    'cell0_recording2',
                    'cell2_recording0',
                    'cell3_recording0',
                    'cell4_recording0',
                    'cell5_recording0'],
    '20080628_R4': ['cell0_recording0',
                    'cell2_recording0',
                    'cell3_recording0',
                    'cell5_recording0',
                    'cell6_recording0',
                    'cell7_recording0',
                    'cell9_recording0',
                    'cell12_recording0',
                    'cell13_recording0'],
    '20080628_R6': ['cell1_recording0',
                    'cell2_recording0',
                    'cell4_recording0',
                    'cell5_recording0',
                    'cell6_recording0',
                    'cell7_recording0',
                    'cell8_recording0',
                    'cell10_recording0']
}

In [4]:
ret_baseline_srs = []
ret_uoi_srs = []
ret_baseline_r2s = []
ret_uoi_r2s = []
ret_baseline_bics = []
ret_uoi_bics = []

# iterate over retina files
for retina, recordings in retinas.items():
    results = h5py.File(ret1_path + retina + '.h5', 'r')
    # iterate over recordings
    for cell_recording in recordings:
        # calculate selection ratios
        lasso_strf = results[cell_recording]['lasso/strfs'][:]
        ret_baseline_srs.append(np.count_nonzero(lasso_strf) / lasso_strf.size)
        uoi_strf = results[cell_recording]['uoi_lasso_bic/strfs'][:]
        ret_uoi_srs.append(np.count_nonzero(uoi_strf) / uoi_strf.size)

        # extract predictive performance
        ret_baseline_r2s.append(results[cell_recording]['lasso/r2s_test'][:].max())
        ret_uoi_r2s.append(results[cell_recording]['uoi_lasso_bic/r2s_test'][:].max())

        # extract model parsimony
        ret_baseline_bics.append(results[cell_recording]['lasso/bics'][:].min())
        ret_uoi_bics.append(results[cell_recording]['uoi_lasso_bic/bics'][:].min())

    results.close()
    
ret_baseline_srs = np.array(ret_baseline_srs)
ret_uoi_srs = np.array(ret_uoi_srs)
ret_baseline_r2s = np.array(ret_baseline_r2s)
ret_uoi_r2s = np.array(ret_uoi_r2s)
ret_baseline_bics = np.array(ret_baseline_bics)
ret_uoi_bics = np.array(ret_uoi_bics)

In [5]:
ecog_fits = h5py.File(ecog_path, 'r')
ecog_baseline = ecog_fits['lasso']
ecog_uoi = ecog_fits['uoi_lasso_bic']
ecog_baseline_srs = np.mean(calculate_selection_ratio(ecog_baseline['tuning_coefs']), axis=0)
ecog_baseline_r2s = np.mean(ecog_baseline['r2'], axis=0)
ecog_baseline_bics = np.mean(ecog_baseline['BIC'], axis=0)
ecog_uoi_srs = np.mean(calculate_selection_ratio(ecog_uoi['tuning_coefs']), axis=0)
ecog_uoi_r2s = np.mean(ecog_uoi['r2'], axis=0)
ecog_uoi_bics = np.mean(ecog_uoi['BIC'], axis=0)

## Selection Ratios

In [6]:
print('Retina')
print('------')
baseline_median = np.median(ret_baseline_srs)
uoi_median = np.median(ret_uoi_srs)

print('Baseline: %0.3f +/- %0.3f' %(baseline_median, ss.iqr(ret_baseline_srs) / 2))
print('UoI: %0.3f +/- %0.3f' %(uoi_median, ss.iqr(ret_uoi_srs) / 2))
print('Reduction Factor: ', baseline_median / uoi_median)
print('WSRT: ', ss.wilcoxon(ret_baseline_srs, ret_uoi_srs))
print('Cohens d: ', np.mean(ret_baseline_srs - ret_uoi_srs) / np.std(ret_baseline_srs))

Retina
------
Baseline: 0.084 +/- 0.020
UoI: 0.017 +/- 0.004
Reduction Factor:  4.984615384615386
WSRT:  WilcoxonResult(statistic=0.0, pvalue=2.7015948487105512e-05)
Cohens d:  2.2062408532240037


In [7]:
print('ECoG')
print('----')
baseline_median = np.median(ecog_baseline_srs)
uoi_median = np.median(ecog_uoi_srs)
print('Baseline: %0.3f +/- %0.3f' %(baseline_median, ss.iqr(ecog_baseline_srs) / 2))
print('UoI: %0.3f +/- %0.3f' %(uoi_median, ss.iqr(ecog_uoi_srs) / 2))
print('Reduction Factor: ', baseline_median / uoi_median)
print('WSRT: ', ss.wilcoxon(ecog_baseline_srs, ecog_uoi_srs))
print('Cohens d: ', np.mean(ecog_baseline_srs - ecog_uoi_srs) / np.std(ecog_baseline_srs))

ECoG
----
Baseline: 0.625 +/- 0.116
UoI: 0.250 +/- 0.114
Reduction Factor:  2.5
WSRT:  WilcoxonResult(statistic=0.0, pvalue=1.3651775993403239e-22)
Cohens d:  1.9830660065038948


## Predictive Performance

In [8]:
print('Retina')
print('------')
baseline_median = np.median(ret_baseline_r2s)
uoi_median = np.median(ret_uoi_r2s)

print('Baseline: %0.3f +/- %0.3f' %(baseline_median, ss.iqr(ret_baseline_r2s) / 2))
print('UoI: %0.3f +/- %0.3f' %(uoi_median, ss.iqr(ret_uoi_r2s) / 2))
print('Median Delta: ', np.median(baseline_median - uoi_median))
print('WSRT: ', ss.wilcoxon(ret_baseline_r2s, ret_uoi_r2s))
print('Cohens d: ', np.mean(ret_baseline_r2s - ret_uoi_r2s) / np.std(ret_baseline_r2s))

Retina
------
Baseline: 0.028 +/- 0.013
UoI: 0.028 +/- 0.013
Median Delta:  -0.0005507204601976623
WSRT:  WilcoxonResult(statistic=50.0, pvalue=0.007439203931621479)
Cohens d:  -0.04884007062708874


In [9]:
print('ECoG')
print('----')
baseline_median = np.median(ecog_baseline_r2s)
uoi_median = np.median(ecog_uoi_r2s)

print('Baseline: %0.3f +/- %0.3f' %(baseline_median, ss.iqr(ecog_baseline_r2s) / 2))
print('UoI: %0.3f +/- %0.3f' %(uoi_median, ss.iqr(ecog_uoi_r2s) / 2))
print('Median Delta: ', np.median(baseline_median - uoi_median))
print('WSRT: ', ss.wilcoxon(ecog_baseline_r2s, ecog_uoi_r2s))
print('Cohens d: ', np.mean(ecog_baseline_r2s - ecog_uoi_r2s) / np.std(ecog_baseline_r2s))

ECoG
----
Baseline: 0.042 +/- 0.034
UoI: 0.041 +/- 0.033
Median Delta:  0.00144526528523177
WSRT:  WilcoxonResult(statistic=345.0, pvalue=3.6003479763987104e-19)
Cohens d:  0.04635893999089997


## Bayesian Information Criterion

In [10]:
print('Retina')
print('------')
baseline_median = np.median(ret_baseline_bics)
uoi_median = np.median(ret_uoi_bics)

print('Baseline: %0.3f +/- %0.3f' %(baseline_median, ss.iqr(ret_baseline_bics) / 2))
print('UoI: %0.3f +/- %0.3f' %(uoi_median, ss.iqr(ret_uoi_bics) / 2))
print('Median Delta: ', np.median(baseline_median - uoi_median))
print('WSRT: ', ss.wilcoxon(ret_baseline_bics, ret_uoi_bics))
print('Cohens d: ', np.mean(ret_baseline_bics - ret_uoi_bics) / np.std(ret_baseline_bics))

Retina
------
Baseline: -1646606.843 +/- 44936.489
UoI: -1647261.080 +/- 44857.249
Median Delta:  654.237336314749
WSRT:  WilcoxonResult(statistic=0.0, pvalue=2.7015948487105512e-05)
Cohens d:  0.005140666493852359


In [11]:
print('ECoG')
print('----')
baseline_median = np.median(ecog_baseline_bics)
uoi_median = np.median(ecog_uoi_bics)

print('Baseline: %0.3f +/- %0.3f' %(baseline_median, ss.iqr(ecog_baseline_bics) / 2))
print('UoI: %0.3f +/- %0.3f' %(uoi_median, ss.iqr(ecog_uoi_bics) / 2))
print('Median Delta: ', np.median(baseline_median - uoi_median))
print('WSRT: ', ss.wilcoxon(ecog_baseline_bics, ecog_uoi_bics))
print('Cohens d: ', np.mean(ecog_baseline_bics - ecog_uoi_bics) / np.std(ecog_baseline_bics))

ECoG
----
Baseline: 4370.767 +/- 2495.118
UoI: 4351.323 +/- 2493.153
Median Delta:  19.444203214014124
WSRT:  WilcoxonResult(statistic=194.0, pvalue=8.309387284435575e-21)
Cohens d:  0.0043749519583627374
